In [67]:
import pandas as pd
import numpy as np

In [8]:
path_death = 'causes_of_death.csv'

df_death = pd.read_csv(path_death)
df_death.head()

,Year,ZIP Code,Causes of Death,Count,Location
0,1999,90002,SUI,1,"(33.94969, -118.246213)"
1,1999,90005,HOM,1,"(34.058508, -118.301197)"
2,1999,90006,ALZ,1,"(34.049323, -118.291687)"
3,1999,90007,ALZ,1,"(34.029442, -118.287095)"
4,1999,90009,DIA,1,"(33.9452, -118.3832)"


In [4]:
path_income = 'irs_income_by_zip_code.csv'

df_income = pd.read_csv(path_income)
df_income.head(2)

,STATE,ZIPCODE,Number of returns,Adjusted gross income (AGI),Avg AGI,Number of returns with total income,Total income amount,Avg total income,Number of returns with taxable income,Taxable income amount,Avg taxable income
0,AL,0,2022380,105089761,51.963410,2022380,106420533,52.621433,1468370,67850874,46.208295
1,AL,35004,4930,255534,51.832454,4930,258024,52.337525,4020,163859,40.760945


In [19]:
df_death_final = df_death[['ZIP Code','Causes of Death','Count']]
df_death_final

,ZIP Code,Causes of Death,Count
0,90002,SUI,1
1,90005,HOM,1
2,90006,ALZ,1
3,90007,ALZ,1
4,90009,DIA,1
5,90009,LIV,1
6,90009,OTH,1
7,90010,STK,1
8,90010,CLD,1
9,90010,DIA,1


In [18]:
df_income_final = df_income[['ZIPCODE', 'Avg AGI']]
df_income_final

,ZIPCODE,Avg AGI
0,0,51.963410
1,35004,51.832454
2,35005,38.905152
3,35006,47.400000
4,35007,53.687073
5,35010,45.492428
6,35014,41.742236
7,35016,46.865007
8,35019,37.536559
9,35020,26.102407


In [78]:
df_max_Crime = grouped.groupby(['ZIP Code']).aggregate(np.max)
df_max_Crime

,Causes of Death,Count
ZIP Code,,
90001,SUI,1007
90002,SUI,1181
90003,SUI,1449
90004,SUI,1181
90005,SUI,670
90006,SUI,1063
90007,SUI,651
90008,SUI,1593
90009,SUI,2


In [86]:
idx = df_death.groupby(['ZIP Code'])['Count'].transform(max) == df_death['Count']

In [99]:
di = df_death[idx]

df_u = di.drop_duplicates(subset=['ZIP Code', 'Count'], keep='first', inplace=False)
df_final = df_u.loc[df_u['Count'] > 20]
df_final = df_final[['ZIP Code', 'Causes of Death', 'Count']]
df_final

,ZIP Code,Causes of Death,Count
185173,93647,HTD,21
185353,91935,CAN,21
185391,93921,HTD,21
185534,95019,HTD,21
185567,96013,HTD,21
185654,94704,HTD,21
185881,93924,HTD,21
186009,95315,HTD,21
186022,96101,OTH,21
186108,94930,CAN,21


In [87]:
zip_count_death = df_death['ZIP Code'].nunique()
zip_count_death

1785